In [1]:
import json
import matplotlib.pyplot as plt

import bluepyopt as bpopt
import bluepyopt.ephys as ephys

import model
import evaluator
import time

import sys

from pathlib import Path

%matplotlib notebook

In [2]:
probe = None #model.define_electrode(probe_type="linear")
model_name = 'hallermann' # "hallermann"
morph_modifier = 'hallermann'

In [3]:
cell = model.create(model=model_name, morph_modifier=morph_modifier, release=True)

param_names = [param.name for param in cell.params.values() if not param.frozen]

fitness_protocols = evaluator.define_protocols(
    model_name,
    feature_set="multiple",
    feature_file="hay_model/features_list.json",
    electrode=probe,
    protocols_with_lfp=None
)

fitness_calculator = ephys.objectivescalculators.ObjectivesCalculator([])

sim = ephys.simulators.LFPySimulator(cell, cvode_active=True, electrode=probe)

eva = ephys.evaluators.CellEvaluator(                    
        cell_model=cell,                                                       
        param_names=param_names,                                                    
        fitness_protocols=fitness_protocols,                                        
        fitness_calculator=fitness_calculator,                                      
        sim=sim)

In [4]:
fitness_protocols

{'bAP': <bluepyopt.ephys.protocols.SweepProtocol at 0x7ff813d33be0>,
 'BAC': <bluepyopt.ephys.protocols.SweepProtocol at 0x7ff813d444c0>,
 'EPSP': <bluepyopt.ephys.protocols.SweepProtocol at 0x7ff813d44a30>,
 'CaBurst': <bluepyopt.ephys.protocols.SweepProtocol at 0x7ff813d46730>,
 'Step1': <bluepyopt.ephys.protocols.SweepProtocol at 0x7ff813d49490>}

In [5]:
t_start = time.time()
responses = eva.run_protocols(protocols=fitness_protocols.values(), param_values={})
print(f"Elapsed time: {time.time() - t_start}")

NEURON: syntax error
 near line 10224
 soma[0] all.append()
            ^


Exception: Traceback (most recent call last):
  File "/gpfs/bbp.cscs.ch/project/proj38/home/damart/LFPy/BluePyOpt/bluepyopt/ephys/mechanisms.py", line 103, in instantiate
    isec.insert(self.suffix)
ValueError: argument not a density mechanism name.

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/gpfs/bbp.cscs.ch/project/proj38/home/damart/LFPy/BluePyOpt/bluepyopt/ephys/protocols.py", line 171, in _run_func
    cell_model.instantiate(sim=sim)
  File "/gpfs/bbp.cscs.ch/project/proj38/home/damart/LFPy/BluePyOpt/bluepyopt/ephys/models.py", line 681, in instantiate
    mechanism.instantiate(sim=sim, icell=self.icell)
  File "/gpfs/bbp.cscs.ch/project/proj38/home/damart/LFPy/BluePyOpt/bluepyopt/ephys/mechanisms.py", line 105, in instantiate
    raise ValueError(str(e) + ': ' + self.suffix)
ValueError: argument not a density mechanism name.: CaH


In [ ]:
fig, axes = plt.subplots(len(responses), figsize=(10, 20), squeeze=False)
for index, (resp_name, response) in enumerate(sorted(responses.items())):
    axes[index, 0].plot(response['time'], response['voltage'], label=resp_name)
    axes[index, 0].set_title(resp_name)
fig.tight_layout()
fig.show()